In [65]:
import psycopg2
from configparser import ConfigParser
import pandas as pd

import common

In [66]:
import warnings
warnings.filterwarnings("ignore")

In [67]:
csv_save_path = "F:\\MSc Project\\temp_sets_100\\"



## Connect to PostgreSQL

In [68]:
def postgresql_config(filename='config.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [69]:
def test_postgresql_connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = postgresql_config()

        print('------Test Connection------')

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)

        # create a cursor
        cur = conn.cursor()

        # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)

        # close the communication with the PostgreSQL
        cur.close()
        print('Database connection closed.')
        # Successfully connected, return True
        print('Successfully connected!')
        return True
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
        print('------Test Finished------')

In [70]:
test_postgresql_connect()

------Test Connection------
Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',)
Database connection closed.
Successfully connected!
------Test Finished------


True

---

## Querry data from PostgreSQl

### Get patients data

In [71]:
def get_patients():
    """ query data from the patients table """
    conn = None
    try:
        params = postgresql_config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()

        cur.execute("SELECT * from patients")
        print("The number of parts: ", cur.rowcount)
        row = cur.fetchone()

        while row is not None:
            print(row)
            row = cur.fetchone()

        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [72]:
# Select some patients as sample
patient_num = 100

sql = "SELECT * FROM patients p order by random() limit " + str(patient_num)
conn = psycopg2.connect(**postgresql_config())

In [73]:
# Execute the sql and form the result as DataFrame
random_patients_df = pd.read_sql(sql, conn)

In [74]:
# Save patient sample
random_patients_df.to_csv(csv_save_path + "patients.csv", index=None)

In [75]:
# Pick Subject_id
patients_subject_ids = random_patients_df['subject_id']
patients_subject_ids

0     44346
1     22903
2      9279
3     20915
4     42124
      ...  
95    95603
96    31552
97    14758
98    26277
99    90756
Name: subject_id, Length: 100, dtype: int64

---

### Define some methods

In [76]:
# create query scripts

def create_query_scripts(table_name):
    return "SELECT * FROM " + table_name + " WHERE subject_id = "

In [77]:
# Query data from database
def select_data(subject_ids, sql):
    '''
    Query data according to the subject from table
    :param subject_ids:
    :param sql:
    :return:
    '''

    # Create the connection object
    conn = None

    try:
        # Get connection
        params = postgresql_config()
        conn = psycopg2.connect(**params)
        # Create a empty DataFrame
        table_df = pd.DataFrame()
        # Iterate subject_ids and query data about patients
        for subject_id in subject_ids:
            new_sql = sql + str(subject_id)
            # Query data and transform to DataFrame form
            select_result = pd.read_sql(new_sql, conn)

            # The first cycle, initial the DataFrame
            if table_df.empty:
                table_df = select_result.copy()
            # DataFrame was initialed, just append the data
            else:
                table_df = pd.concat([table_df, select_result], axis=0)

        return table_df
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [78]:
def query_others(patients_subject_ids, table_name_list):
    '''
    Pass in a list containing table names, traverse the list, and query data from each table
    :param patients_subject_ids: patients id
    :param table_name_list: table list
    '''
    for table_name in table_name_list:
        print("Start querying: " + table_name)
        # Query data
        sample = select_data(patients_subject_ids, create_query_scripts(table_name))
        # Save sample to csv file
        sample.to_csv(csv_save_path + table_name + ".csv", index=None)
    print("All done!" )

## Start querying

In [79]:
table_list = ['admissions', 'callout', 'icustays', 'chartevents', 'outputevents', 'inputevents_mv', 
            'procedureevents_mv', 'cptevents', 'labevents', 'microbiologyevents', 'procedures_icd']

In [80]:
query_others(patients_subject_ids, table_list)

Start querying: admissions
Start querying: callout
Start querying: icustays
Start querying: chartevents
Start querying: outputevents
Start querying: inputevents_mv
Start querying: procedureevents_mv
Start querying: cptevents
Start querying: labevents
Start querying: microbiologyevents
Start querying: procedures_icd
All done!
